# install the current pyroSAR development branch

```shell
sudo apt-get install git
python -m pip install git+https://github.com/johntruckenbrodt/pyroSAR.git@feature/auxdatahandler
```

# prepare several DEM files in Gamma format for a Sentinel-1 scene

documentation:
- [pyroSAR.identify](https://pyrosar.readthedocs.io/en/latest/pyroSAR.html#pyroSAR.drivers.identify)
- function dem_autocreate is not documented on readthedocs yet, but a docstring can be called in Python: `help(dem_autocreate)`
- the download of TanDEM-X 90m DEMs requires a registration, see [here](https://geoservice.dlr.de/web/dataguide/tdm90) for details; once registered the credentials can be passed to function dem_autocreate; for the other DEM options they are ignored

In [ ]:
import os
from pyroSAR import identify
from pyroSAR.gamma.srtm import dem_autocreate

filename = 'S1A_IW_GRDH_1SDV_20150426T190111_20150426T190136_005659_00741C_FFB7.zip'

outdir = ''

id = identify(filename)

demfiles = {}
for demType in ['AW3D30', 'SRTM 1Sec HGT', 'SRTM 3Sec', 'TDX90m']:
    dem_id = demType.replace(' ', '-')
    dem_base = 'demfile_gamma_{}'.format(dem_id)
    dem = os.path.join(outdir, dem_base)
    print(demType)
    with id.bbox() as bbox:
        dem_autocreate(geometry=bbox, demType=demType, outfile=dem, 
                       username='username', password='password')
    demfiles[dem_id] = dem

# process the scene with the different DEMs

documentation:
- [pyroSAR.gamma.geocode](https://pyrosar.readthedocs.io/en/latest/pyroSAR.html#pyroSAR.gamma.geocode)

In [ ]:
from pyroSAR.gamma import geocode

# parallelize the Gamma commands
os.environ['OMP_NUM_THREADS'] = '6'

for dem_id, dem_file in demfiles.items():
    scenedir = os.path.join(outdir, '{}_{}'.format(id.outname_base(), dem_id))
    geocode(scene=id, dem=dem_file,
            tempdir=os.path.join(scenedir, 'process'), outdir=scenedir,
            targetres=90, scaling='db')